# Analysis of WeRateDogs Twitter

## Table of Contents
- [Introduction](#intro)
- [Gathering Data](#Gather)
- [Assessing Data](#Assess)
- [Cleaning Data](#Clean)



<a id='intro'></a>
## Introduction

WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc,WeRateDogs has over 4 million followers and has received international media coverage.
The main purpose in this project is to gather data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it , plus showcase them through analyses and visualizations 

<a id='Gather'></a>
## Gathering Data
In this project I gather data from a varity of sorces :
1. The WeRateDogs Twitter archive
2. The tweet image predictions,This file is hosted on Udacity's servers and should be downloaded programmatically 
3. Twitter API & JSON: Each tweet's retweet count and favorite ("like") count 


In [5]:
# Load all packages
import pandas as pd
import requests
import os

import json

In [4]:
#Read twitter archive csv file
twitter_archeive = pd.read_csv('twitter-archive-enhanced.csv')

In [11]:
url= 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
folder_name = "C:\\Users\\Ahmed Ezz\\Desktop\\Wrangle and analyze data project"
response = requests.get(url)
with open(os.path.join(folder_name,url.split('/')[-1]),mode='wb') as file:
    file.write(response.content)
image_prediction = pd.read_csv('image-predictions.tsv',sep='\t')